In [1]:
import re
import json
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from bs4 import BeautifulSoup
from pyvirtualdisplay import Display
import csv

#---------------------- ----------------------------- -------------
# Prevent chrome browser from opening while browsing 
display = Display(visible=0, size=(800, 600))
display.start()
# Set up webdriver with chrome engine 
s=Service("/home/revo/chromedriver/chromedriver")
driver = webdriver.Chrome(service=s)
#driver = webdriver.Chrome("/home/revo/chromedriver/chromedriver")

#----------------------------------------------------------------------------

# IIoT devices manufacturing companies
iiot_list = ('Linx', 'Johnson', 'GridConnect', 'Impinj')

# WSN devices
wsn_list =  {'ANT', 'WiSense', 'AquisGrain', 'Arduino BT', 'AS-XM1000', 'AVRaven', 'AWAIRS1', 'BEAN', 'BPart', 'BSN Node v2', 'BSN Node v3', 'BTnode', 'BTnode rev3', 'CENS Medusa MK2', 'CIT', 'CoSeN', 'Cricket', 'Dalian', 'Dot', 'DSRPN', 'DSYS25', 'Egs', 'Ember', 'EmberNet', 'EnOcean', 'TCM120', 'ENS', 'EPIC', 'e-Watch', 'eXtreme', 'EYES', 'eyesIFXv2.1', 'FemtoNode', 'XSM', 'FireFly', 'FireFly3', 'Fleck', 'G-Node', 'GWNode', 'Hermes', 'Hogthrob', 'iBadge', 'iCubes', 'IECAS', 'iMote1', 'iMote2', 'Indriya_DP_01A11', 'Indriya_DP_03A14', 'Indriya_DP_03A20', 'Iris', 'iSense', 'JN5121', 'Kmote-B', 'LEAP', 'Lotus', 'LTC5800-WHM', 'LTP5900-WHM', 'LTP5901-WHM', 'LTP5902-WHM', 'M12', 'Mantis', 'Nymph', 'MASN', 'MediMesh', 'Mica', 'Mica2', 'Mica2Dot', 'MicaZ', 'Micromote', 'MITes', 'Monnit Wit', 'mPlatform', 'MTM-CM3000-MSP', 'MTM-CM3300-MSP', 'MTM-CM4000-MSP', 'Mulle', 'panStamp'}

# ICS devices manufacturing companies
ics_list = ('GE Digital', 'Bosch', 'Siemens', 'Mitsubishi', 'Schneider', 'ABB', 'Schweitzer', 'General Electric', 'Wind River')

# IoT devices manufacturing companies
iot_list = ('Amazon Echo', 'Google Home', 'Verizon', 'digi', 'Particle', 'SmartCare', 'Edimax', 'ZigBee', 'Z-Wave', 'E2 Max', 'E7 Beacon', 'P1 Plus', 'August Doorbell', 'August Smart', 'Kuri Mobile', 'Belkin WeMo', 'Footbot Air', 'Nest Smoke', 'Nest T3021US', 'Philips Hue', 'Bitdefender BOX', 'Ring Doorbell', 'WeMo Insight', 'Logitech Harmony', 'Particle Photon', 'NETGEAR Orbi', 'Canary', 'Cinder', 'Nespresso Prodigio', 'Netatmo', 'Samsung SmartThings', 'Sonos', 'Wink', 'Apple Watch', 'Garmin Forerunner', 'Peloton', 'Google Cardboard', 'Samsung Gear')

embedded_systems = iiot_list+ics_list+iot_list
embedded_iot = iot_list
embedded_ics = ics_list
embedded_iiot = iiot_list

cve_pointer = "" # Points to the CVE where the word is found
index = 0 # The line where the word is found in the CVE description
cves = {} # Stores the found CVEs using the word lists
vlist = {} # Stores the word and the line number where it found in CVE database
count = 0
CVEList = {} # Contains the unique CVEs our goal list but the serial number is in a form 'Name: CVE-?????'
cvess = [] # Contains the CVE serial number  in a form 'CVE-????'
delim = '======================================================' # The delimiter between cves
cvecount = 0 # Number of exist CVEs from 2015 to 2021
rc = 0

In [2]:
#---------------------------------------------------------------
with open("cve.txt", "r") as file:
    lines = file.readlines()

    for i in range(0, len(lines)):
        index += 1
        if re.search(delim, lines[i]):
            cve_pointer = lines[i+1]
            cvecount += 1
        for word in embedded_systems:
            if re.search("Reference:", lines[i]):
                rc += 1
                break                   
            elif re.search(r'\b'+word+r'\b', lines[i]):
                vlist[index] = word
                # Store CVES in cves array
                cves[index] = cve_pointer
                break
#print(cvecount)
print(len(cves))

700


In [3]:
# Get ride of repetitions
for key, value in cves.items():
    CVEList[cves[key]] = 'A'
print(len(CVEList))
for i in CVEList.keys():
    cvess.append(i[6:-1])

#print(cvess)

600


In [5]:
non_embedded = open("non_embedded.txt", "w+")
array = []
with open("filtredCVEs.txt", "r") as fcve:
    ff = fcve.readlines()
    for i in range(len(ff)):
        array.append(ff[i].strip())
    for entry in cvess:
        if entry not in array:
            non_embedded.write(entry+'\n')

In [7]:
#!usr/bin/python



'''for j in cvess:
    cvelist.writelines([j, '\n'])'''
#=====================================================================================

    #for k in exist:
        #print(table[k])

#=====================================================================================

'''table = {} # Stors all exist {'CVE': 'CWE'} in the json file
cwes = [] # Stores the CWE of only CVEs in the cvess list
non_exist_cves = [] # Stors non-defined in NVD CWEs

with open("nvdcve-1.1-2015.json", "r") as f:
    data = json.load(f)
    for i in data['CVE_Items']:
        for j in range(0, len(i['cve']['problemtype']['problemtype_data'])):
            for k in range(0, len(i['cve']['problemtype']['problemtype_data'][j]['description'])):
                # 'CVE':'CWE'
                 table[i['cve']['CVE_data_meta']['ID']] = i['cve']['problemtype']['problemtype_data'][j]['description'][k]['value']
    #print(len(table))
    for cve in table:
        if cve in cvess:
            exist_cves.append(cve)
    print(len(exist_cves))

    noninfo = 0
    for cve in cvess:
        try:
            cwes.append(table[cve])
            if re.search('NVD', table[cve]):
                noninfo += 1
        except KeyError:
            non_exist_cves.append(cve)
    #print(len(cwes))
    #print(noninfo)
    #print(len(non_exist_cves))

#=======================================================================================================================================

with open('cve_cwe.csv', 'w+') as f:
    write =  csv.writer(f)
    for j in cvess:
        try:
            driver.get("https://cwe.mitre.org/data/definitions/"+table[j][4:]+".html")
            content = driver.page_source
            soup = (content)
            for div in soup.findAll('div', attrs={'style':'overflow:auto;'}):
                write.writerow([j, div.h2.string])
        except KeyError:BeautifulSoup
            continue

f.close()
'''
# Let store the CVEs in an html file to make the parsing more easier
link = "https://nvd.nist.gov/vuln/detail/"

arraylist = []
htmlfile = open("CVEhtml.html", "a+")
for entry in cvess:
    arraylist.append(entry)

for value in arraylist:
     htmlfile.write('<a href='+link+value+'>'+value+'</a></br>')

file.close()

#==========================================================================================================================================

700
600


In [7]:
import pandas as pd
from pandas.plotting import scatter_matrix
#import seaborn as sns
#import matplotlib.pyplot as plt

datalist = []
data = pd.read_csv("cve_cwe.csv")
data.describe()
cweplot = data.groupby('CWE')['CVE'].count()

#sns.factorplot(x="CWE", y ="Num", data=data, kind="bar", size=3)
#plt.show()
#%matplotlib inline
#cweplot.plot(kind='bar', stacked=True, title="The title of my graph")

#scatter_matrix(data)
#data['CWE'].value_counts()
#uniques = pd.unique(data['CWE'])
#with open("cwedata.csv", "w+") as fcsv:
#    writef =  csv.writer(fcsv)
#    for i in uniques:
#        writef.writerow([data.groupby('CWE')['CVE'].count()[i], i])
#print (summ)

KeyError: 'CWE'

In [20]:
from selenium import webdriver
from bs4 import BeautifulSoup
from pyvirtualdisplay import Display

display = Display(visible=0, size=(800, 600))
display.start()
driver = webdriver.Chrome("/home/revo/chrome-driver/chromedriver")

# List to store CWE-Names
cwe_names = []

driver.get("https://cwe.mitre.org/data/definitions/259.html")

content = driver.page_source
soup = BeautifulSoup(content)
for div in soup.findAll('div', attrs={'style':'overflow:auto;'}):
    print(div.h2.string)


CWE-259: Use of Hard-coded Password


In [61]:
import csv
import re

iotList = []
icsList = []
common  = []

iot_f = open("iot_cwedata.csv", "r")
ics_f = open("ics_cwedata.csv", "r")
diff_iot = open("iot_diff", "w+")
diff_ics = open("ics_diff", "w+")
iot = csv.reader(iot_f)
ics = csv.reader(ics_f)

for i in ics:
    icsList.append(i[1])
for j in iot:
    iotList.append(j[1])

for k in list(set(iotList).difference(icsList)):
    diff_iot.writelines([k, '\n'])